# Google Colab Connect

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import statements 

In [ ]:
from sklearn.cluster import KMeans
import numpy as np
import cv2
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
import math
from sklearn.metrics.pairwise import pairwise_distances
import time
from PIL import Image
import scipy.spatial.distance as d
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure 
import json
import datetime
import copy
from PIL import Image as im 
import joblib

In [ ]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [ ]:
import scipy.cluster.vq as vq
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision

# Saving and Loading function 

In [ ]:
# Saving and Loading models using joblib 
def save(filename, obj):
  with open(filename, 'wb') as handle:
      joblib.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load(filename):
  with open(filename, 'rb') as handle:
      return joblib.load(filename)

# Argumnets and data Path

In [ ]:
class Args():
  def __init__(self,data_path,dataset,ranking_save_folder):
    self.data_save_path = data_path
    self.pca = os.path.join(data_path,'pca.pkl') 
    self.img_features_global = os.path.join(data_path,dataset+'_all_img_features_fpn_layer.pkl')
    self.img_names =  os.path.join(data_path,dataset+'_all_img_names_fpn_layer.pkl')
    self.ground_truth = os.path.join(data_path,dataset +'_GT_final')
    self.ranking_save_folder = ranking_save_folder
    self.img_dict = os.path.join(data_path,'data.pkl')
    self.datasetName = dataset
    self.all_original_images = '/content/drive/MyDrive/SUB/PROJECTS/IR_final/datasets/all_images/'+ dataset +'/'

In [ ]:
# # my_paths = Args('/content/drive/MyDrive/IR_final/models/final_dl_model/instre/others','instre','/content/drive/MyDrive/SUB/PROJECTS/IR_final/models/final_dl_model/instre/ranking/')
# # my_paths = Args('/content/drive/MyDrive/IR_final/models/final_dl_model/oxford/others/','oxford','/content/drive/MyDrive/SUB/PROJECTS/IR_final/models/final_dl_model/oxford/ranking/')
# my_paths = Args('/content/drive/MyDrive/IR_final/models/final_dl_model/paris/others/','paris','/content/drive/MyDrive/SUB/PROJECTS/IR_final/models/final_dl_model/paris/ranking/')
# # my_paths = Args('/content/drive/MyDrive/IR_final/models/final_dl_model/sculpture/others/','sculpture','/content/drive/MyDrive/SUB/PROJECTS/IR_final/models/final_dl_model/sculpture/ranking/')

In [ ]:
# my_paths = Args('/content/drive/MyDrive/SUB/PROJECTS/IR_final/models/final_dl_model/instre/others/','instre','/content/drive/MyDrive/SUB/PROJECTS/IR_final/models/final_dl_model/instre/ranking/')
# my_paths = Args('/content/drive/MyDrive/SUB/PROJECTS/IR_final/models/final_dl_model/oxford/others/','oxford','/content/drive/MyDrive/SUB/PROJECTS/IR_final/models/final_dl_model/oxford/ranking/')
# my_paths = Args('/content/drive/MyDrive/SUB/PROJECTS/IR_final/models/final_dl_model/paris/others/','paris','/content/drive/MyDrive/SUB/PROJECTS/IR_final/models/final_dl_model/paris/ranking/')
my_paths = Args('/content/drive/MyDrive/SUB/PROJECTS/IR_final/models/final_dl_model/sculpture/others/','sculpture','/content/drive/MyDrive/SUB/PROJECTS/IR_final/models/final_dl_model/sculpture/ranking/')

In [ ]:
data_save_path = my_paths.data_save_path 
# pca = load(my_paths.pca)
img_features_global = load(my_paths.img_features_global)
img_names = load(my_paths.img_names)
ground_truth = load(my_paths.ground_truth)
ranking_save_folder_path = my_paths.ranking_save_folder
# img_dict = load(my_paths.img_dict)
datasetName = my_paths.datasetName


In [ ]:
all_original_images = my_paths.all_original_images
all_original_images

'/content/drive/MyDrive/SUB/PROJECTS/IR_final/datasets/all_images/sculpture/'

In [ ]:
len(img_features_global), len(img_names), np.array(img_features_global).shape

(6340, 6340, (6340, 256))

# Pca model generation 

In [ ]:
# according to paper after sum pooling features are normalized and whiten (using pca)
# pca is performed as the same number of dim as given by last conv layer kernel  
def learn_transform(feats, dimension, path):
    feats = normalize(feats)
    pca = PCA(dimension,whiten=True)
    pca.fit(feats)
    save(path + "pca.pkl",pca)
    print("pca model is saved ..")

In [ ]:
# learn_transform(img_features_global, 256, my_paths.data_save_path)

# Utility functions 

## model generation 

In [ ]:
def model_generation():
  device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

  # pre trained model 
  model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
  model.eval()
  model.to(device)
  return model 

def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

## Evaluation 

### MAP functions 

In [ ]:
# labelled array is an array of 0,1,0,1  and rev_idx gives number of relevant docs to be considered for mean avg precision
def find_MAP(labelled_arr, rev_idx):
  count = 0
  rel = 0
  true_pre = 0
  rel_enc = 0
  for label in labelled_arr:
    count = count + 1
    if rel_enc < rev_idx:
      if label == 1:
        rel_enc = rel_enc + 1
        rel = rel + 1 
        true_pre = true_pre + rel/count 
      # print("true Precision for relevant query for label", rel/count )
  #print("count", count)  
  if rel == 0:
    return 0
  else:
    avgPre = true_pre/(rel)
    
    return avgPre

In [ ]:
# original list contains extension .jpg


def get_map(query_image_name, top_result_names):
  def get_good_ok_images(query_image_name):
    # print(query_image_name)
    if '.' in query_image_name:
      query_image_name = query_image_name[:query_image_name.index('.')]
      # print(query_image_name)
    if query_image_name in ground_truth.keys():
      obj = ground_truth.get(query_image_name, dict())
      # print(obj)
      goodNames = []
      if 'good' in obj.keys():
        goodNames = list(obj.get('good', []))
      else:
        # print('no good names...')
        pass
      if 'ok' in obj.keys():
        goodNames.extend(obj.get('ok', []))
      else:
        # print('no ok names...')
        pass
      if 'junk' in obj.keys():
        # goodNames.extend(obj.get('junk', []))
        pass
      else:
        # print('no ok names...')
        pass
      if 'ignore' in obj.keys():
        # goodNames.extend(obj.get('junk', []))
        pass
      else:
        # print('no ok names...')
        pass
        
      goodNames = list(set(goodNames))
      # print(goodNames)
      return goodNames
    else:
      # print('query image not present...')
      pass
      return None

  names = get_good_ok_images(query_image_name)
  scores = [0]*len(top_result_names)

  if names is None:
    return scores

  for idx, i in enumerate(top_result_names):
    if i in names:
      scores[idx] = 1
  return scores

### DCG functions

In [ ]:
# original list contains extension .jpg


def DGC_get_map(query_image_name, top_result_names):
  def get_good_ok_images(query_image_name):
    # print(query_image_name)
    if '.' in query_image_name:
      query_image_name = query_image_name[:query_image_name.index('.')]
      # print(query_image_name)
    if query_image_name in ground_truth.keys():
      obj = ground_truth.get(query_image_name, dict())
      # print(obj)
      goodNames = []
      okNames = []
      junkNames = []
      ignoreNames = []

      if 'good' in obj.keys():
        goodNames = list(obj.get('good', []))
      else:
        # print('no good names...')
        pass
      if 'ok' in obj.keys():
        okNames = list(obj.get('ok', []))
      else:
        # print('no ok names...')
        pass
      if 'junk' in obj.keys():
        junkNames = list(obj.get('junk', []))
        pass
      else:
        # print('no ok names...')
        pass
      if 'ignore' in obj.keys():
        ignoreNames = list(obj.get('ignore', []))
        pass
      else:
        # print('no ok names...')
        pass
        
      goodNames = list(set(goodNames))
      # print(goodNames)
      return goodNames, okNames, junkNames, ignoreNames
    else:
      # print('query image not present...')
      pass
      return None

  goodNames, okNames, junkNames, ignoreNames = get_good_ok_images(query_image_name)
  scores = [0]*len(top_result_names)

  if goodNames is None and okNames is None and junkNames is None and ignoreNames is None:
    return scores

  for idx, i in enumerate(top_result_names):
    if i in goodNames:
      scores[idx] = 3
    elif i in okNames:
      scores[idx] = 2
    elif i in junkNames:
      scores[idx] = 1
    elif i in ignoreNames:
      scores[idx] = 1
  return scores

In [ ]:
def compute_dcg_at_k(r, k, m='f1'):
    # get k values 
    r = np.asfarray(r)[:k]
    if r.size:
        if m == 'f1':
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif m == 'f2':
            return np.sum((np.power(2, r) - 1) / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('not valid.')
    return 0

def compute_ndcg_at_k(r, k, m='f1'):
    dcg_max = compute_dcg_at_k(sorted(r, reverse=True), k, m)
    if not dcg_max:
        return 0.
    return compute_dcg_at_k(r, k, m) / dcg_max  

def compute_idcg_at_k(r, k, m='f1'):
    dcg_max = compute_dcg_at_k(sorted(r, reverse=True), k, m)
    if not dcg_max:
        return 0.
    else:
      return dcg_max
# it computes the max dcg value given relevance list and which formula to use 
def compute_idcg(relevance, alternate='f1'):

    if relevance is None or len(relevance) < 1:
        return 0.0
    r = np.asarray(relevance).copy()
    r.sort()
    return dcg(r[::-1], alternate) 

# required functions 

# it computes the dcg value given relevance list and which formula to use 
# f1 -> formula of DCG
# f2 -> alternative formula of DCG 
def compute_dcg(relevance, alternate='f1'):
    # a list of all the relevance scores provided 
    if relevance is None or len(relevance) < 1:
        return 0.0
    rel = np.asarray(relevance)
    p = len(rel)

    if alternate == 'f2':
        #An alternative formulation DCG
        log2i = np.log2(np.asarray(range(1, p + 1)) + 1)
        return ((np.power(2, rel) - 1) / log2i).sum()
    else:
        log2i = np.log2(range(2, p + 1))
        return rel[0] + (rel[1:] / log2i).sum()

# print("DGC", compute_dcg_at_k(relData, 50, m='f1'))
# print("iDGC", compute_idcg_at_k(relData, 50, m='f1'))
# print("nDGC", compute_ndcg_at_k(relData, 50, m='f1') )


## Ranker functions 

### Get Query vectors 

In [ ]:
# groud truth is a dictionary with query names as keys and values containing (annotation and good, ok, bad images)
def get_query_vectors(ground_truth,dimension, img_names,img_features_global ):
  """
    
  """
  query_names = list(ground_truth.keys())
  query_feats = np.zeros((len(query_names),dimension))
  # removing .jpg from names in img_names
  img_names_without_ext = [i.split('.')[0] for i in img_names ]

  i = 0
  
  for query in query_names:
      
      query_feats[i,:] = img_features_global[np.where(np.array(img_names_without_ext) == query)[0][0]]
      
      i+=1

  query_feats = normalize(query_feats)
  return query_feats


### Distance computing function 

In [ ]:
def get_distances(query_feats,img_features_global,dist_type):
    distances = pairwise_distances(query_feats,img_features_global,dist_type, n_jobs=-1)
    return distances

### Saving rankings to disk 

In [ ]:
def write_rankings(final_scores, img_names, save_folder):
  i = 0
  query_names = list(ground_truth.keys())
  for query in query_names:
      scores = final_scores[i,:]
      ranking = np.array(img_names)[np.argsort(scores)]
      save(save_folder + query + "_ranking_.pkl",ranking)
      i+=1

### Main Ranking function (Generates ranking one for all images) 

In [ ]:
def rank(img_features_global,dist_type,ground_truth,img_names,save_folder):

  query_feats = get_query_vectors(ground_truth,256, img_names,img_features_global )
  print("Computing distances...")
  t0 = time.time()

  final_scores = get_distances(query_feats,img_features_global,dist_type)
  print( "Done. Time elapsed", time.time() - t0)

  print("Writing rankings to disk...")
  t0 = time.time()
  write_rankings(final_scores,img_names,save_folder)
  print("Done. Time elapsed", time.time() - t0)

In [ ]:
# rank(img_features_global,'cosine',ground_truth,img_names, ranking_save_folder_path)